# PySpark SQL

This notebook is based on https://github.com/jadianes/spark-py-notebooks

It turns around the concept of Data Frame and using SQL language to query them.

### Getting the data files
In this notebook, we will use the reduced dataset (10 percent) provided for the KDD cup 1999, containing nearly half million network interactions. The file is provided as a Gzip file that we will download locally

In [1]:
import urllib.request
data = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")


### Create a PySpark session in a Jupyter Notebook

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SQL-PySpark').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

### Create a RDD from a file

In [3]:
data_file = './kddcup.data_10_percent.gz'
raw_data = sc.textFile(data_file)

### Getting a Data Frame

A Spark DataFrame is a distributed collection of data organized into named columns. 

The entry point into all SQL functionality in Spark is the SQLContext class. To create a basic instance, all we need is a SparkContext reference. Since we are running Spark in shell mode (using pySpark) we can use the global context object sc for this purpose.

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Inferring the schema
With a SQLContext, we are ready to create a DataFrame from our existing RDD. But first we need to tell Spark SQL the schema in our data.

Spark SQL can convert an RDD of Row objects to a DataFrame. Rows are constructed by passing a list of key/value pairs as kwargs to the Row class. The keys define the column names, and the types are inferred by looking at the first row. Therefore, it is important that there is no missing data in the first row of the RDD in order to properly infer the schema.

In our case, we first need to split the comma separated data, and then use the information in KDD's 1999 task description to obtain the column names.

In [5]:
from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [11]:
# Create RDD of Row 
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.printSchema()

# SQL queries
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()

root
 |-- duration: long (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: long (nullable = true)
 |-- dst_bytes: long (nullable = true)

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

